In [1]:
# Hugging face testing
from transformers import AutoTokenizer, TFAutoModelForMaskedLM
import tensorflow as tf

tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")

model = TFAutoModelForMaskedLM.from_pretrained("bert-base-chinese")

/userhome/cs2/u3507348/anaconda3/envs/tensorflow_2_11/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-04-12 09:56:53.352078: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-12 09:57:35.367740: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-12 09:57:35.433069: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read fro

In [3]:
def testModel(sentence, k = 5):
    # Tokenize the input sentence
    inputs = tokenizer(sentence, return_tensors='tf')

    # Get the position of the [MASK] token
    mask_token_index = tf.where(inputs['input_ids'] == tokenizer.mask_token_id)[0, 1]

    # Generate model predictions for the masked token
    outputs = model(inputs['input_ids'], attention_mask=inputs['attention_mask'])
    predictions = outputs.logits[0, mask_token_index]

    # Convert the predictions to probabilities
    probabilities = tf.nn.softmax(predictions, axis=-1)

    # Get the top-k predicted tokens and their probabilities
    top_k_values, top_k_indices = tf.math.top_k(probabilities, k=k)
    predicted_tokens = tokenizer.convert_ids_to_tokens(top_k_indices.numpy())

    # Print the top-k predicted tokens and their probabilities
    print(f"Top-{k} predictions:")
    for i in range(k):
        print(f"{predicted_tokens[i]}: {top_k_values[i].numpy()}")
        
# Define the input sentence with [MASK] token
sentence = "今天的天氣很清[MASK]舒服"
testModel(sentence)

Top-5 predictions:
爽: 0.6484012007713318
新: 0.16201545298099518
涼: 0.08563422411680222
朗: 0.0348283126950264
淨: 0.006423946004360914


In [19]:
# Try fine tune it with own data
from datasets import load_dataset

dataset = load_dataset("csv", data_files="../dataset/set_01_02_new.csv", split='train')
dataset = dataset.train_test_split(test_size=0.2)

print(dataset)

dataset = dataset.filter(lambda example: all(example[key] is not None for key in example))

train_data = dataset['train']
test_data = dataset['test']

print(train_data[0])
print(test_data[0])

Found cached dataset csv (/userhome/cs2/u3507348/.cache/huggingface/datasets/csv/default-84069fde21d8afc3/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 4155
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 1039
    })
})


{'label': 0, 'text': 'great trip india bring light everyone project everyone lucky see smile bye abiola'}
{'label': 1, 'text': 'congrats number mobile number g videophones r call number videochat wid mate play java game dload polyph music noline rentl'}


In [21]:
# Set up tokenizer
tokenized_train_data = tokenizer(train_data["text"], return_tensors="np", padding=True)

# Tokenizer returns a BatchEncoding, but we convert that to a dict for Keras
tokenized_train_data = dict(tokenized_train_data)

print(tokenized_train_data)


{'input_ids': array([[  101, 10512, 12358, ...,     0,     0,     0],
       [  101,  9386,  8334, ...,     0,     0,     0],
       [  101,   162,  9570, ...,     0,     0,     0],
       ...,
       [  101, 12605,  8458, ...,     0,     0,     0],
       [  101,   150,  8278, ...,     0,     0,     0],
       [  101,  9064,  8179, ...,     0,     0,     0]]), 'token_type_ids': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), 'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])}
